In [ ]:
#@title Downloading helper files...<p>(Run this cell first!)\n",
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/priv-sec/privacy/master/exercise7_data/binary_model.py", "binary_model.py"),
urllib.request.urlretrieve("https://raw.githubusercontent.com/priv-sec/privacy/master/exercise7_data/cifar10_model.py", "cifar10_model.py")

## Membership Inference Attack (Salem et al.)

Paper Link: https://arxiv.org/pdf/1806.01246.pdf

You have to implement the adversary 1 scenario : TOWARDS MODEL INDEPENDENT MEMBERSHIP INFERENCE ATTACKS(ADVERSARY1). The single shadow model has the same structure and uses the same algorithms as the target model.

**1) Preprocess the Cifar10 dataset**

**2) Train target model**

**3) Train shadow model**

**4) Create attack dataset**

**5) Train attack model**

**6) Evaluation**

Note:
- The binary model outputs a single numerical value which can then be rounded to obtain the label '0' or '1'
- Use torch.save(model.state_dict(), "./model.pt") to safe the weights of your model
- Use model.load_state_dict(torch.load("./model.pt")) to load the weights of your model

## Preprocess the Cifar10 dataset

In [ ]:
import torch
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

from cifar10_model import Cifar10Net, train_cifar10_model
from binary_model import BinaryNet, train_binary_model

In [ ]:
TRAIN_BATCHSIZE = 32
TEST_BATCHSIZE = 1000

EPOCHS_TARGET = 1 # [1, 10, 30, 50]
EPOCHS_SHADOW = 40
EPOCHS_ATTACKER = # 1-20 #

In [ ]:
def get_cifar10_dataset(root_dir="./data"):
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

    return train_dataset, test_dataset


def split_dataset(dataset):

    train_target_indices, test_target_indices, train_shadow_indices, test_shadow_indices, *_ = \
                 torch.utils.data.random_split(dataset, [len(dataset)//5]*5)

    train_target = torch.utils.data.DataLoader(train_target_indices, **train_kwargs)
    test_target = torch.utils.data.DataLoader(test_target_indices, **test_kwargs)
    train_shadow = torch.utils.data.DataLoader(train_shadow_indices, **train_kwargs)
    test_shadow = torch.utils.data.DataLoader(test_shadow_indices, **test_kwargs)

    return (train_target, test_target), (train_shadow, test_shadow)

In [ ]:
cuda_available = torch.cuda.is_available()
device = torch.device("cuda" if cuda_available else "cpu")

train_kwargs = {
    'batch_size': TRAIN_BATCHSIZE,
    'shuffle': True
    }

test_kwargs = {
    'batch_size': TEST_BATCHSIZE,
    'shuffle': True
    }

if cuda_available:
    cuda_kwargs = {
        'num_workers': 2,
        'pin_memory': True,
        'shuffle': True
        }
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

In [ ]:
train_dataset, test_dataset = get_cifar10_dataset()

(train_target_loader, test_target_loader), (train_shadow_loader, test_shadow_loader) = split_dataset(train_dataset + test_dataset)

## Target Model

In [ ]:
target_model = Cifar10Net().to(device)
# TODO: Train target model; Please use the provided training method #

## Shadow Model

In [ ]:
shadow_model = Cifar10Net().to(device)
# TODO: Train shadow model; Please use the provided training method #

## Create Attack Dataset

In [ ]:
def getTopk(data, top):
    """
    Returns the top k maximum entries of the given data.
    """
    
    return # TODO #


# TODO: Create softmax layer with the correct dimension #

def predict_(model, data_loader):
    """
    Iterates the given data_loader and collects the predicition vectors
    of the given model in a list.
    Note: It is essential for this attack to apply the softmax function 
          DIRECTLY AFTER receiving the prediction vectors of the Cifar10 models
    """

    preds = list()
    with torch.no_grad():
        # TODO: Store predictions in <preds> #
        # Hint: Be aware of the batches! #
    return preds

In [ ]:
shadow_model.eval()

# IN -> Predictions for members (Label 1)
shadow_preds_IN = # TODO #
shadow_labels_IN = # TODO #

# OUT -> Predictions for non-members (Label 0)
shadow_preds_OUT = # TODO #
shadow_labels_OUT = # TODO #

# ALL = IN + OUT
shadow_preds = np.concatenate([shadow_preds_IN, shadow_preds_OUT])
shadow_labels = np.concatenate([shadow_labels_IN, shadow_labels_OUT])

# Top 3 prediction values for every data point
shadow_preds = # TODO #

In [ ]:
# Create attack dataset -> train (and test)

size_test_data = 1000

attack_dataset = torch.utils.data.TensorDataset(torch.Tensor(shadow_preds[size_test_data:]).type(torch.FloatTensor), torch.Tensor(shadow_labels[size_test_data:]).type(torch.LongTensor) )
train_attacker_loader = torch.utils.data.DataLoader(attack_dataset, **train_kwargs)

attack_dataset_test = torch.utils.data.TensorDataset(torch.Tensor(shadow_preds[:size_test_data]).type(torch.FloatTensor), torch.Tensor(shadow_labels[:size_test_data]).type(torch.LongTensor) )
test_attacker_loader = torch.utils.data.DataLoader(attack_dataset_test, **test_kwargs)

## Attack Model

In [ ]:
attack_model = BinaryNet().to(device)
# TODO: Train attack model; Please use the provided training method#

## Evaluation

In [ ]:
target_model.eval()

# IN -> Predictions for members (Label 1)
target_preds_IN = # TODO #
target_labels_IN = # TODO #

# OUT -> Predictions for non-members (Label 0)
target_preds_OUT = # TODO #
target_labels_OUT = # TODO #

# ALL = IN + OUT
target_preds = np.concatenate([target_preds_IN, target_preds_OUT])
target_labels = np.concatenate([target_labels_IN, target_labels_OUT])

# Top 3 prediction values for every data point
target_preds = # TODO # 

In [ ]:
# Create evaluation dataset
eval_dataset = torch.utils.data.TensorDataset(torch.Tensor(target_preds).type(torch.FloatTensor), torch.Tensor(target_labels).type(torch.LongTensor) )
train_eval_loader = torch.utils.data.DataLoader(eval_dataset, **test_kwargs)

In [ ]:
attack_model.eval()

with torch.no_grad():
    # TODO: Iterate evaluation dataset and calculate the accuracy on the target model #

accuracy = # TODO #

print("Accuracy", accuracy)